# Skript for TMS Data Cleaning - Part 1

<strong><em>Important: This is a guide, which helps and explains you the data cleaning we where doing before this Hack-a-thon. There are parts you can and sometimes should directly copy and paste. You won't be able to copy the whole notebook and run it within your project.</em></strong>

## Creating the Client Connection to the Cloud Object Storage and the "smart-city-live-vehicle-positions" bucket

The following code cell can be automatically inserted trough the Notebook UI. To do so, click on the data button (top right corner) there you find the *files* and *connections* tab. Go to the *connection* as we want to create a client to our Cloud Object Storage. 

There you will find the Connection which we created before. Click "insert to code" and choose the "StreamingBody object" option. After that there will open a pop up which showes you the folder structure of your underlying cloud bucket. Choose the right folders and subfolders until you end up in the last subfolder, that contains all the .json files we need. Choose one file and click *Select*. Next you will see a code cell, inserted automatically, that looks like this one except it contains the correct api-keys etc.

> It doesn't matter which .json you will choose, because we will later on only use the created client object to access more then only one .json file.

In [ ]:
# @hidden_cell


# This connection object is used to access your data and contains your credentials or project token.
# You might want to remove those credentials before you share your notebook.

import os
import types
import pandas as pd
import ibm_boto3
from botocore.client import Config
import logging

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.

connection_TMS_trafficData_client = ibm_boto3.client(
    service_name='s3',
    ibm_api_key_id='api-key',
    ibm_service_instance_id='service-instance-id',
    ibm_auth_endpoint='https://iam.cloud.ibm.com/identity/token',
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-de.cloud-object-storage.appdomain.cloud'
)

body = connection_TMS_trafficData_client.get_object(Bucket='smart-city-tms', Key='topics/digitraffic_tms/partition=0/digitraffic_tms+0+0011067000.json')['Body'].read()
# add missing __iter__ method, so pandas accepts body as file-like object 

if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# Since JSON data can be semi-structured and contain additional metadata, it is possible that you might face an error during data loading.
# Refer to the documentation of 'pandas.read_json()' and 'pandas.io.json.json_normalize' for more possibilities to adjust the data loading.
# pandas documentation: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
# and http://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html

pd.read_json(body, lines=True)

After we sucsessfully created the client (your's might be named differently than `Cloud_Object_Storage_Connection_client`, either change it in the cell above or keep in mind to change the name whenever the client is used) we now need a function that can read/get/access more than one .json file. 

As we know, the data is saved in a S3 object style. The following cell shows the function we use to access files over a given timespan in which they were written to the storage. (Don't stress about the function and how it works in detail 😉) Just copy and paste it.


In [2]:
import argparse
import boto3
import dateutil.parser
import logging
import pytz
from collections import namedtuple

import pandas as pd
from datetime import datetime, timezone, timedelta



logger = logging.getLogger(__name__)


Rule = namedtuple('Rule', ['has_min', 'has_max'])
last_modified_rules = {
    Rule(has_min=True, has_max=True):
        lambda min_date, date, max_date: min_date <= date <= max_date,
    Rule(has_min=True, has_max=False):
        lambda min_date, date, max_date: min_date <= date,
    Rule(has_min=False, has_max=True):
        lambda min_date, date, max_date: date <= max_date,
    Rule(has_min=False, has_max=False):
        lambda min_date, date, max_date: True,
}

def get_s3_objects(s3, bucket, prefixes=None, suffixes=None, last_modified_min=None, last_modified_max=None):
    
    if last_modified_min and last_modified_max and last_modified_max < last_modified_min:
        raise ValueError(
            "When using both, last_modified_max: {} must be greater than last_modified_min: {}".format(
                last_modified_max, last_modified_min
            )
        )
    # Use the last_modified_rules dict to lookup which conditional logic to apply
    # based on which arguments were supplied
    last_modified_rule = last_modified_rules[bool(last_modified_min), bool(last_modified_max)]

    if not prefixes:
        prefixes = ('',)
    else:
        prefixes = tuple(set(prefixes))
    if not suffixes:
        suffixes = ('',)
    else:
        suffixes = tuple(set(suffixes))

    kwargs = {'Bucket': bucket}

    for prefix in prefixes:
        kwargs['Prefix'] = prefix
        while True:
            # The S3 API response is a large blob of metadata.
            # 'Contents' contains information about the listed objects.
            resp = s3.list_objects_v2(**kwargs)
            for content in resp.get('Contents', []):
                last_modified_date = content['LastModified']
                if (
                    content['Key'].endswith(suffixes) and
                    last_modified_rule(last_modified_min, last_modified_date, last_modified_max)
                ):
                    yield content

            # The S3 API is paginated, returning up to 1000 keys at a time.
            # Pass the continuation token into the next response, until we
            # reach the final page (when this field is missing).
            try:
                kwargs['ContinuationToken'] = resp['NextContinuationToken']
            except KeyError:
                break

## Defining the Timespan, which jsons/S3 objects will be collected

As introduced right before this, we are able to access data from TMS trough our client and define a timespan in which we want to get the data.
Further we need Metadata, which might not be written within the choosen timewindow. So leave the metastarttime as it is.

We decided to create three variables:
`dateloading`,
`starttime`,
`endtime`, to create the timespan we were talking about. 

> Even for a few hours the data that has been collected can sum up to 1.000.000+ rows. So to get a feeling of the data cleaning process it is more than enough to create a small timespan (one hour). Also do remember that it is possible, if you change date and time, that there is no data available.

The use of metastarttime will explane itself later on.

In [3]:
dateloading = "2022-02-25"
#dateloading = str(os.environ['DATE'])
starttime = datetime.fromisoformat(dateloading + ' 10:00:00.000+00:00')
endtime = datetime.fromisoformat(dateloading + ' 12:00:00.000+00:00')
metastarttime = datetime.fromisoformat("2022-02-20" + ' 00:00:00.000+00:00')


Trough the `Cloud_Object_Storage_Connection` and with the usage of the defined method `get_s3_objects` we are now able to access our s3 objects, that were written within the defined timewindow.

In [4]:
objs=get_s3_objects(s3=connection_TMS_trafficData_client,bucket="smart-city-tms", last_modified_min=starttime, last_modified_max=endtime)

The variable `objs` is now filled with these s3 Objects, which isn't a format we can really work with in terms of the final data in form of .json. So it requires one more step to extract the wanted data into our pandas.DataFrame.

## Reading the Vehicle Positions from the variable objs and store them into a DataFrame

To receive our data, we use the variable `objs` and iterate trough every `obj` that it contains. We have defined an empty DataFrame (`df_tms`) which will be filled step by step with the data we want to extract. To do so, we have to use our client again. We use `.get_object()`, give it the exact Bucket we want to access and the Key to our data, which is stored in each `obj['Key']['Body']` and read the lines we "find" there which are in the .json format. Around that call, we use `pd.read_json()` to extract the data from the json. This input (now in the form of a DataFrame) now will be appended to the `df_tms`. After extracting all the jsons from all the given `obj` out of `objs`, we finally reset the index, drop the old one.

We then make a first quick step of Data Cleaning by throwing out the rows, which don't contain a roadStationId since we can't really work with these values when the identification is missing.

In [5]:
df_tms = pd.DataFrame()
for obj in objs:
    df_tms = df_tms.append(pd.read_json(connection_TMS_trafficData_client.get_object(Bucket='smart-city-tms', Key=obj['Key'])['Body'].read(), lines=True))
    
df_tms = df_tms.reset_index()
df_tms = df_tms[df_tms['roadStationId'].notna()]

Now, let's take a look if that worked out

In [ ]:
df_tms

How many rows x columns do we have at hand? 

In [6]:
df_tms.shape

(780280, 15)